<a href="https://colab.research.google.com/github/calde97/Deep_Learning_Challenge/blob/main/first_challenge/Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
SEED = 1234 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Download data from kaggle to drive

In [ ]:
#I forced the upgrade since I had some problems with the previous API. To apply just once
!pip install --upgrade --force-reinstall --no-deps kaggle
!kaggle -v

**From kaggle to colab** <br>
We need to use the API from kaggle to allow the download from colab. We donwload the token as json and then save it into a hidden folder kaggle.

In [ ]:
#Upload json kaggle
from google.colab import files
files.upload()

In [ ]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
 ! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets list  #just curiosity for all the ds available on kaggle

Donwload Zip from kaggle competion

In [ ]:
!kaggle competitions download -c artificial-neural-networks-and-deep-learning-2020


 96% 398M/413M [00:03<00:00, 176MB/s]
100% 413M/413M [00:03<00:00, 140MB/s]


Create directory challenge1 to unzip our ds

In [ ]:
!mkdir /content/drive/My\ Drive/challenge1

In [ ]:
!unzip /content/artificial-neural-networks-and-deep-learning-2020.zip -d /content/drive/My\ Drive/challenge1

# Preprocess the data for tensorflow flow_from_directory

We now try to get the data clean

In [ ]:
%cd /content/drive/My\ Drive/challenge1/MaskDataset

In [ ]:
import json
import numpy as np

with open('train_gt.json') as file:
  data = json.load(file)
items = data.items()
image_target_list = list(items)
targets = np.array(image_target_list)[:, 1]
targets = np.unique(targets)

In [ ]:
%cd /content/drive/My\ Drive/challenge1/MaskDataset/training/2
!ls | wc -l

In [ ]:
import os 
import shutil
cwd = os.getcwd()
training_path = os.path.join(cwd, 'training')

for label in targets:
  if not os.path.exists(os.path.join(training_path, str(label))):
    os.mkdir(os.path.join(training_path, str(label)))

for image, label in image_target_list:
  shutil.move(os.path.join(training_path, image), os.path.join(training_path, str(label), image))



# Building the algorithm

## Image Augmentation

Image Augmentation (Zoom, brightness, small rootation, horizontal flip) the normalization. Since I don't want to create 2 separate folders I'm building 2 imagedatagenerator that slows from the same directory. I set the validation split equal to both. Also the SEED must be equal to take the same pictures

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os 

%cd /content/drive/My Drive/challenge1/MaskDataset
cwd = os.getcwd()
training_path = os.path.join(cwd, 'training')


VALIDATION_SPLIT = 0.15
bs = 8

#Creating Training generator with augmentation
train_image_data_generator = ImageDataGenerator(rescale=1/255.,
                                                horizontal_flip=True,
                                                rotation_range=30,
                                                brightness_range=[0.2, 1.2],
                                                zoom_range=[0.7, 1.3],
                                                validation_split=VALIDATION_SPLIT)

train_data_generator = train_image_data_generator.flow_from_directory(training_path,
                                                                      batch_size=bs,
                                                                      seed=SEED,
                                                                      subset='training',
                                                                      class_mode='categorical')


#Creating Validation generator
validation_image_data_generator = ImageDataGenerator(rescale=1/255.,
                                                     validation_split=VALIDATION_SPLIT)


validation_data_generator = validation_image_data_generator.flow_from_directory(training_path,
                                                                                batch_size=bs,
                                                                                seed=SEED,
                                                                                subset='validation',
                                                                                class_mode='categorical')


/content/drive/My Drive/challenge1/MaskDataset
Found 841 images belonging to 3 classes.
Found 841 images belonging to 3 classes.


In [ ]:
image_width = 256
image_height = 256
channels = 3
classes_number = 3

train_dataset = tf.data.Dataset.from_generator(lambda : train_data_generator,  ##There was a mistake here
                                               output_types=(tf.float32, tf.float16),
                                               output_shapes=([None, image_width, image_height, channels], [None, classes_number]))
train_dataset = train_dataset.repeat()

validation_dataset = tf.data.Dataset.from_generator(lambda : validation_data_generator,
                                            output_types=(tf.float32, tf.float16),
                                            output_shapes=([None, image_width, image_height, channels], [None, classes_number]))
validation_dataset = validation_dataset.repeat()



## Building the baseline MODEL

Trying to use the model created in the 3rd lab w/o augmentation.
###Feature extraction CNN + MLP


In [ ]:
start_f = 8
depth = 5 

model = tf.keras.Sequential()

for i in range(depth):
  if i == 0:        
    input_shape = [image_width, image_height, channels]
  else:
    input_shape = [None]
  
  model.add(tf.keras.layers.Conv2D(filters=start_f,
                                   kernel_size=(3,3),
                                   padding='same',
                                   strides=(1,1),
                                   input_shape=input_shape))
  
  model.add(tf.keras.layers.ReLU())
  model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2)))
  start_f *= 2


In [ ]:
#Building the classifier (MLP)
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units=512,
                                activation='relu'))
model.add(tf.keras.layers.Dense(units=classes_number, activation='softmax'))
#model.summary()

### Model fit

In [ ]:
#Work in progess ahah

# Create results file for test images (Not completed... waiting for the model to try )

In [ ]:
import os
from datetime import datetime

def create_csv(results, results_dir='./'):

    csv_fname = 'results_'
    csv_fname += datetime.now().strftime('%b%d_%H-%M-%S') + '.csv'

    with open(os.path.join(results_dir, csv_fname), 'w') as f:

        f.write('Id,Category\n')

        for key, value in results.items():
            f.write(key + ',' + str(value) + '\n')

In [ ]:
from PIL import Image
import numpy as np

results = {}

test_path = os.path.join(cwd, 'test')
names = os.listdir(test_path)


for img_name in names:
  img = Image.open(os.path.join(test_path, img_name)).convert('RGB')
  nimg = np.array(img).shape
  nimg = np.expand_dims(nimg, 0)
  nimg = nimg / 255.
  softmax_output = model.predict(nimg)
  prediction = tf.argmax(softmax_output, 1)
  class_label = prediction.numpy()[0]
  results[img_name] = class_label

create_csv(results,results_dir="/content/drive/My Drive/challenge1/MaskDataset")

# Not Useful

In [ ]:
#Not useful for now 
import matplotlib.pyplot as plt
%matplotlib inline 

def plot_image(augmented_img):
  augmented_img = np.array(augmented_img)
  augmented_img = augmented_img * 255
  plt.imshow(np.uint8(augmented_img)) 
  plt.plot()

#img, tg = train_data_generator.next()
#plot_image(img[2])